In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance

np.random.seed(2205)

In [6]:
df = pd.read_csv("sub_info_and_features.csv")

df["Gender_int"] = (df["Gender"] == "F").astype(int)

eeg_features = [col for col in df.columns if col not in ["participant_id", "Gender", "Age", "Group", "MMSE"]]
eeg_df = df[eeg_features]

In [7]:
X = eeg_df
y = df["Group"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Logistic Regression with L1

In [9]:
log_reg = LogisticRegression(penalty="l1", solver="saga", C=1.0, max_iter=1000)
log_reg.fit(X_scaled, y)

if len(np.unique(y)) == 2:
    coef = np.abs(log_reg.coef_[0])
else:
    coef = np.abs(log_reg.coef_).mean(axis=0)

log_importance = pd.DataFrame({
    "feature": eeg_df.columns,
    "importance": coef
}).sort_values("importance", ascending=False)

print("Top 10 features (linear regression):")
print(log_importance.head(10))

Top 10 features (linear regression):
                         feature  importance
6                    Delta/Alpha    0.618517
17        frontal_sample_entropy    0.307637
47                 occipital_DFA    0.270291
20      frontal_spectral_entropy    0.254451
23     frontal_hjorth_complexity    0.231131
38       parietal_sample_entropy    0.182373
35  temporal_permutation_entropy    0.174003
24        central_sample_entropy    0.167704
46             occipital_higuchi    0.167257
0                          Delta    0.157616


C:\Users\falco\miniconda3\envs\dm\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Random Forest

In [11]:
rf = RandomForestClassifier(n_estimators=100, random_state=2205)
rf.fit(X, y)

rf_importance = pd.DataFrame({
    "feature": eeg_df.columns,
    "importance": rf.feature_importances_
}).sort_values("importance", ascending=False)

print("Top 10 features (random forest):")
print(rf_importance.head(10))

Top 10 features (random forest):
                        feature  importance
20     frontal_spectral_entropy    0.035912
6                   Delta/Alpha    0.035480
45     occipital_sample_entropy    0.034862
47                occipital_DFA    0.032777
51  occipital_hjorth_complexity    0.031765
17       frontal_sample_entropy    0.031684
50    occipital_hjorth_mobility    0.027763
10                      cD2_RMS    0.026797
4                   Theta/Alpha    0.026377
0                         Delta    0.025916
